# Context managerek "with" szerkezettel

Eredeti cikk:  
https://realpython.com/python-with-statement/


Erőforrás kezelés általában 3 lépésből áll:
* megnyitás
* kezelés/felhasználás
* lebontás

Ezt kétféle úton szokás megvalósítani:
* **try ... finally** szerkezettel - szabadon és helyzetre szabottan használható
* **with ...** szerkezettel - szabványosított felület sokféle erőforráshoz


## A try ... finally megoldás

ezzel a szerkezettel biztosítjuk a fájl lezárását akkor is, ha az írás közben hiba történne

In [3]:
# Biztonsagos faljnyitas
file = open("hello.txt", "w")

try:
    file.write("Hello, World!")
finally:
    # Igy tuti lezarjuk a fajlt a vegen
    file.close()

Az esetleges hibákat még egy **except:** résszel le is kezelhetjük

In [5]:
# Biztonsagos faljnyitas
file = open("hello.txt", "w")

try:
    file.write("Hello, World!")
except Exception as e:
    print(f"A fáljírás közben {e}  hiba történt.")
finally:
    # Igy tuti lezarjuk a fajlt a vegen
    file.close()


## A with ... megoldás

A **with** kifejezés egy context managert-t biztosít mindazon erőforások számára, amik megvalósítják a *context management protocol*-t. 
Ez főleg abból áll, hogy a használandó erőforrás a következő metódusokkal rendelkezik:
* \_\_enter\_\_() - ez hívódik a megnyitáskor
* \_\_exit\_\_() - ez hívódik automatikusan a lezáráskor, a **with** blokkból kilépéskor. 

A lényeg, hogy a kezelendő erőforrás - nagyon sok esetben ez fájl kiírás vagy olvasás - megnyitásáról és lezárásáról a context manager protocol gondoskodik. Így biztosan nem marad véletlenül se megnyitva semmi. 

A **with** használatával tehát biztonságosabban és tisztábban kezelhetjük az erőforrásainkat, automatán intézi a nyitást és zárást minden körülmények között, így segít elkerülni az erőforráspazarlást (leaking)



In [ ]:

# ez csak minta, nem fut 
with valamilyen_kifejezés as cél_változó:
    csinálunk_valamit(cél_változó)


A következő lépéseken megyünk végig:
* "valamilyen_kifejezés" meghívása, ami visszaadja a context managert
* A .\_\_enter\_\_() és .\_\_exit\_\_() metódusok elmentése későbbre
* .\_\_enter\_\_() hívása, az eredmény a *cél_változó*ba kerül
* kódblokk végrehajtása - ez jellemzően a *cél_változó* valamilyen manipulálását jelenti

Összességében kevesebb - és jobban olvasható - kód az eredmény. Szinte balzsam a szemnek. 


In [7]:

# a hello.txt-s példa "with" használatával

with open('hello.txt', 'w') as file:
    file.write('Context managerrel irunk.') 


Az *open* egy olyan objektumot ad vissza, aminek van \_\_enter\_\_() és \_\_exit\_\_() metódusa. (TextIOBase)

---
A Python 3.1 óta egyszerre több context managert is fel lehet építeni, mondjuk úgy, hogy egy fáljból olvasunk és egy másikba írunk



In [ ]:
# például így egy bemeneti és egy kimeneti fájl használható egyszerre

with open('input_file.txt','r') as infile, open('output_file.txt','w') as outfile:
    # infile-ból olvasunk
    # valamit teszünk vele
    # outfile-ba írunk
    pass


# a végén mindkét file szépen lezáródik

Fájlok elegáns és Pythonos kezelésére jól használható a **pathlib** modulból a **Path** osztály. Ez egy OS fájl elérési utat reprezentál és a metódusain keresztül lehet fájlokat kezelni. 

Külön figyelmet érdemel az OS-független könyvtárkezelés ( \/, \\, \\\\ és társai), valamint a / operátor, amivel az útvonal darabokat tudjuk összefűzni.(könyvtárba belépés)

Az alábbi példában egy **try ... except** blokkba csomagoljuk a **with** kifejezést, mert így szép és felhasználóbarát üzenetet tudunk kitolni hiba esetén. 

In [17]:
import pathlib
import logging

# létrejön a Path objektum a fájlunkkal
file_path = pathlib.Path("hello.txt")

file_path = pathlib.Path(".") / "hello.txt"

try:
    with file_path.open(mode="w") as file:
        file.write("Hello, World!")
except OSError as error:
    logging.error("%s fájlba írás sikertelen. %s kivétel történt", file_path, error)

## Könyvtárbejárás

Az **os** modul **scandir()** függvénye egy iterátort ad, ami **os.DirEntry()** objektumokból áll. Kifejezetten a hatékony könyvtárbejárás kedvéért.

In [16]:
import os

with os.scandir('.') as entries:
    for i, entry in enumerate(entries):
        print(f"{i}: {entry.name} -> {entry.stat().st_size} bytes")

0: context_managers.ipynb -> 8766 bytes
1: hello.txt -> 13 bytes


## Nagy pontosságú számítások

Ahol a sima float számítási pontossága nem elég, ott segít a **decimal** modul **Decimal** osztálya. Az alábbi példában 42 tizedesig számolunk, amit a **localcontext()** managerben állítunk be. 

In [21]:
from decimal import Decimal, localcontext

with localcontext() as ctx:
    ctx.prec = 42 # itt állítjuk be a pontosságot
    print(  Decimal('1') / Decimal('42') , "ez van a localcontext-ben")
# itt lezáródik a licalcontext()

print(  Decimal('1') / Decimal('42') , "ez van a localcontext nélkül")


0.0238095238095238095238095238095238095238095 ez van a localcontext-ben
0.02380952380952380952380952381 ez van a localcontext nélkül


## Lock kezelés többszálú programoknál

A **with** arra is jó, hogy minden kikért lockot egészen biztosan elengedjünk ott, ahol már nincs rá szükség, anélkül, hogy erről külön gondoskodni kellene. A context manager protocol miatt pont a helyén meghívódik automatikusan a .acquire() és a .release() is. 

In [ ]:
import threading

balance_lock = threading.Lock()

# try ... finally esetén így néz ki
balance_lock.acquire() #mondjuk mintha itt nem lenne kezelve az, hogy nem sikerül a lockot megszerezni...
try:
    # csináljuk amit a lock alatt kell
    pass
finally:
    balance_lock.release()


# with használata esetén
with balance_lock:
    # csináljuk amit lock alatt kell
    pass

## Külső könyvtárak

Egy csomó olyan könyvtári objektum is tud context manager lenni, ami nem része a standard python környezetnek. A **pytest** például ilyen

In [23]:
import pytest

# ez így egy csúnya nullával osztás
1 / 0

ZeroDivisionError: division by zero

In [25]:
import pytest

# így szépen lekezelhető a hiba
with pytest.raises(ZeroDivisionError):
    1 / 0

In [26]:
import pytest

favorites = {"fruit": "apple", "pet": "dog"}
# ez így KeyError lesz
favorites["car"]

KeyError: 'car'

In [29]:
import pytest

favorites = {"fruit": "apple", "pet": "dog"}

# így szépen lekezelhető a KeyError
with pytest.raises(KeyError):
    favorites["car"]

Ezek így csak akkor jók, ha hibára számítunk és tényleg lesz is hiba. Ha a fenti példában *nem* nullával osztunk, akkor hibát kapunk. Ezt lehet kikerülni azzal, hogy *target value*-t is megadunk, hogy abba rakja a **pytest.raises()** az esetleg keletkező excetion-t

In [35]:
import pytest

with pytest.raises(ZeroDivisionError) as exc:
    # 1 / 0  # ezzel a várt eredményt kapja a pytest
    1 / 1  # ezzel meg nem

print( exc.value )
assert str(exc.value) == "division by zero", "Nullával kellene osztani."

Failed: DID NOT RAISE <class 'ZeroDivisionError'>

## Az async with: működés aszinkron módban

Sok IO művelet esetén a várakozás csökkentésére hasznos lehet a konkurrens mód használata, amikor az éppen várakozó (IOwait) kérés alatt másik kérés feldolgozására is lehetőség van.  Erre példa az melékelt weboldal ellenőrző kód (site_checker_v0.py)

* 3. sor: importáljuk amit kell. Az aiohttp lehet, hogy telepítést is igényel.
* 6. sor: a **check()** függvény, amit **async** kulcsszóval definiálunk
* 7. sor: külső **async with**, amiben az **aiohttp.ClientSession()** jön létre.
* 8. sor: belső **async with**, ami az előzőben létrejött session-ön hívja a **get()**-et a paraméterként megadott url-lel
* 9. sor: kiírjuk a válaszban lévő státusz kódot
* 10. sor: hívjuk az aszinkron várakozásra képes **.text()** metódust a válaszon (response) és a **html**-be kerül az eredmény
* 11. sor: kiírjuk az url-t és a doctype-t
* 13. sor: **async** kulcsszóval definiáljuk a **main()** függvényt (így ez is *coroutine* lesz)
* 14. sor: hívjuk az **asyncio.gather()**-t, ami a felsorolt *várakozásra képes (awaitable)* objektumokat futtatja, konkurrensen. 
* 19. sor: az **asyncio.run()** segítségével indítunk egy eseményfeldolgozót (event loop)




In [ ]:
# site_checker_v0.py - ezt a külön fájlból futtassad

import aiohttp
import asyncio

async def check(url):
    async with aiohttp.ClientSession() as session:
        async with session.get(url) as response:
            print(f"{url}: status -> {response.status}")
            html = await response.text()
            print(f"{url}: type -> {html[:17].strip()}")

async def main():
    await asyncio.gather(
        check("https://realpython.org"),
        check("https://pycoders.com"),
    )

asyncio.run(main())

## Házi gyártmányú context managerek

Sok esetben lehet szükség context managerre ott, ahol amúgy gyárilag nincs. Ilyesmik pl:
* nyit - zár
* beállít - resetel
* létrehoz - letöröl
* elindít - leállít

Ezek lehetnek **osztály alapúak**, ehhez meg kell valósítaniuk egy \_\_enter\_\_() és egy \_\_exit\_\_() metódust. A szükséges paraméterek a példában. 

Vagy lehetnek **függvény alapúak**, ehhez a **contextlib.contextmanager** dekorátort lehet használni, ha nem akarunk egy komplett osztályt létrehozni. A függvénynek viszont egy korrekt generátornak kell lennie. 

## Class (osztály) alapú context managerek

In [59]:
# Példa class alapú context managerre 

class HelloContextManager:
     def __enter__(self):
         print("Entering the context...")
         return "Hello, World!"
     def __exit__(self, exc_type, exc_value, exc_tb):
         print("Leaving the context...")
         print(exc_type, exc_value, exc_tb, sep="\n")
         
         # ha exception kezelést akarsz, akkor azt tedd ide.
         # Ha az __exit__() True értékkel tér vissza, akkor a program nem áll le


with HelloContextManager() as hello:
    print('>>>',hello,'<<<')
    # print(hello[100])  # ha a hibára futást próbálnád, akkor élesítsd ezt a sort

print('És innen tovább normálisan...')

Entering the context...
>>> Hello, World! <<<
Leaving the context...
None
None
None
És innen tovább normálisan...


In [73]:
# időmérés context manager által

from time import perf_counter, sleep

class Timer:
    def __enter__(self):
        self.start = perf_counter()
        self.end = 0.0
        # az __enter__() metódus így egy lambda függvényt ad vissza,
        # ez kerül majd a "timer" változóba a with blokkban
        return lambda: self.end - self.start

    def __exit__(self, *args):
        self.end = perf_counter()

with Timer() as timer:
    # valami hosszút csinálunk
    sleep(1)

# itt meghívjuk az __enter__ által visszaadott lambdát, ami az időkülönbséget mondja meg.
timer()

1.0004085000109626

## Függvény alapú context managerek

a **contextlib.contextmanager** dekorátort lehet használni, ha nem akarunk egy komplett osztályt létrehozni. A függvénynek viszont egy korrekt generátornak kell lennie. 

A függvényben legyen egy darab **yield**. Amielőtte van, az lesz az __enter__() rész, amit a yield visszaad, az megy a *with* ... *as* **változó**: szerkezet változójába. Ami a yield után van, az pedig az __exit__() rész. 

In [75]:
# file írás fügvény alapú context managerrel

from contextlib import contextmanager

@contextmanager
def writable_file(file_path):
    file = open(file_path, 'w')
    try:              # csekkoljuk, hogy sikerült-e a file megnyitása
        yield file    # ha igen, akkor a file objektumot visszaadjuk (__enter__ vége)
    finally:
        file.close()  # ha végeztünk, bezárjuk. (ez az __exit__ rész)


with writable_file("hello_ctx_func.txt") as file:
    file.write("Sikerult beleirni.")

In [76]:
# valós idő elfedése context managerrel (tesztelési célra pl)

from contextlib import contextmanager
from time import time

@contextmanager
def mock_time():
    # __enter__ rész kezdete
    global time         # jelezzük, hogy a time objektumot globálisan változtatjuk
    saved_time = time   # elmentjük az eredeti függvény címét
    time = lambda: 42   # time függvény felülírása egy lambdával
    yield               # __enter__ rész vége

    # __exit__ rész
    time = saved_time   # a time függvény helyreállítása

with mock_time():  # itt nincs "as", mert nincs mit manipulálni
    print(f"Teszteléshez elfedett idő: {time()}")

print(f"Normál idő: {time()}")



Teszteléshez elfedett idő: 42
Normál idő: 1678189469.6485324


Kihagyott rész: saját async context manager készítése

A site_checker példaprogramok *nálam* valamiért nem működnek jól.